In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import glob
from matplotlib.ticker import FixedLocator, FixedFormatter


In [41]:

deg = 3
deg_trigo = 2
terms1 = (2*deg+1)*(2*deg+1)
terms3 = (2*deg+1)*(2*deg+1)*(2*deg+1)
terms2 = (deg)*(deg+1)*(2*deg+1)
total_terms = terms1 + terms2
dict1 = [None]*terms1
dict2 = [None]*terms2
dict3 = [None]*terms3
dict = [None]*total_terms

counter = 0
for i in range(-deg, deg+1):
    for j in range(-deg, deg+1):
        dict1[counter] = 'Re^'+str(i) +'phi^'+str(j)
        counter = counter +1

counter = 0
for i in range(1, deg+1):
    for j in range(0, deg+1):
        for k in range(-deg, deg+1):
            dict2[counter] = 'Re^'+str(k) + '(phi-1)^'+str(i) +'theta^'+str(2*j)
            counter = counter +1

counter = 0
for i in range(-deg, deg+1):
    for j in range(-deg, deg+1):
        for k in range(-deg, deg+1):
            dict3[counter] = 'Re^'+str(k) + 'phi^'+str(i) +'theta^'+str(j)
            counter = counter +1


dict = dict1 + dict2
dict = dict3


In [42]:

np.random.seed(142)

path = r'data/ellipsoid_inclined_data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    frame = pd.read_csv(filename, index_col=None, header=0)
    li.append(frame)

df = pd.concat(li, axis=0, ignore_index=True)
df = pd.DataFrame(df, columns= ['Re', 'phi', 'theta', 'Exp C_d'])

frac = 1 # what fraction of data to work at. 1 for 100 data, .8 for 80, .6 for 60, .4 for 40, .2 for 20, .1 for 10 data
shuffled_df = df.sample(frac=frac)

data = shuffled_df.to_numpy()
Re = data[:,0]
phi = data[:,1]
theta = data[:,2]
Cd = data[:,3]
inp = data[:,0:3]



In [43]:
def library(Re, phi, theta, deg, deg_trigo):
    terms1 = (2*deg+1)*(2*deg+1)
    terms2 = deg * (deg+1) * (2*deg+1)
    lib1 = np.zeros((len(Re), terms1))
    lib2 = np.zeros((len(Re), terms2))
    lib = np.zeros((len(Re), total_terms))
    lib1_names = [None]*terms1
    lib2_names = [None]*terms2

    counter = 0
    for i in range(-deg, deg+1):
        for j in range(-deg, deg+1):
            lib1[:,counter] = Re**i * phi**j
            lib1_names[counter] = 'Re^'+str(i) +'phi^'+str(j)
            counter = counter +1

    counter = 0
    for i in range(1, deg+1):
        for j in range(0, deg+1):
            for k in range(-deg, deg+1):
                lib2[:,counter] = Re**k * (phi-1)**i * theta**(2*j)
                lib2_names[counter] = 'Re^'+str(k) + '(phi-1)^'+str(i) +'theta^'+str(2*j)
                counter = counter +1

    lib = np.concatenate((lib1, lib2), 1)
    lib_names = lib1_names + lib2_names

    return lib, lib_names


def library2(Re, phi, theta, deg, deg_trigo):
    terms3 = (2*deg+1)*(2*deg+1)*(2*deg+1)
    lib = np.zeros((len(Re), terms3))
    lib_names = [None]*terms3

    counter = 0
    for i in range(-deg, deg+1):
        for j in range(-deg, deg+1):
            for k in range(0, deg+1):
                lib[:,counter] = Re**i * phi**j * theta**k
                lib_names[counter] = 'Re^'+str(i) + 'phi^'+str(j) +'theta^'+str(k)
                counter = counter +1

    return lib, lib_names


In [44]:

inp_train, inp_test, Cd_train, Cd_test = train_test_split(inp, Cd, test_size=0.2, shuffle=True)
Re_train = inp_train[:,0]
phi_train = inp_train[:,1]
theta_train = inp_train[:,2]
Re_test = inp_test[:,0]
phi_test = inp_test[:,1]
theta_test = inp_test[:,2]

library_train, library_names = library(Re_train, phi_train, theta_train, deg, deg_trigo)
library_test, library_names = library(Re_test, phi_test, theta_test, deg, deg_trigo)

library2_train, library2_names = library2(Re_train, phi_train, theta_train, deg, deg_trigo)
library2_test, library2_names = library2(Re_test, phi_test, theta_test, deg, deg_trigo)


In [45]:
# Re = inp[:,0]
# phi = inp[:,1]
# theta = inp[:,2]

# lib, lib_names = library(Re, phi, theta, deg, deg_trigo)
# lib2, lib2_names = library2(Re, phi, theta, deg, deg_trigo)

# library_train, library_test, Cd_train, Cv_test = train_test_split(lib2, Cd, test_size=0.2, shuffle=False)

# Re_train, Re_test, phi_train, phi_test, theta_train, theta_test, Cd_train, Cd_test = \
#     train_test_split(Re, phi, theta, Cd, test_size=0.2, shuffle=True)


In [50]:

alphas = np.linspace(1e1, 1e5, 10) 
reg = linear_model.LassoCV(fit_intercept=False, alphas=alphas, positive=True, tol=1e-3, max_iter=1000000)

reg.fit(library_train, Cd_train)

Cd_pred = reg.predict(library_test)
train_score = reg.score(library_train, Cd_train)
test_score = reg.score(library_test, Cd_test)
print('alpha = ', reg.alpha_)

print("train score: ",train_score)
print("test score: ",test_score)
print("----")

rmse_train = np.sqrt(mean_squared_error(Cd_train, reg.predict(library_train)))
rmse_test = np.sqrt(mean_squared_error(Cd_test, reg.predict(library_test)))
#percent_error = mean_absolute_percentage_error(Cd_test, reg.predict(library_test))
print("RMSE train: ",rmse_train)
print("RMSE test: ",rmse_test)
print("----")
print("----")

coefs = reg.coef_
learned_dict = list(zip(library_names, coefs))
for i in range(len(learned_dict)):
    if np.abs(learned_dict[i][1]) > 0:
        print(learned_dict[i][0], learned_dict[i][1])

alpha =  10.0
train score:  0.9976466151042969
test score:  0.9971463332618932
----
RMSE train:  195.5987015237021
RMSE test:  242.99458150874145
----
----
Re^-1phi^-1 12.235989036569045
Re^-1phi^0 12.570199193065921
Re^0phi^1 1.901355580699162
Re^-1(phi-1)^1theta^2 0.4958313799820437
Re^0(phi-1)^1theta^2 0.9546361263619828


In [54]:
alphas = np.linspace(1e-1, 1e5, 10) 
reg = linear_model.LassoCV(fit_intercept=False, alphas=alphas, positive=True, tol=1e-4, max_iter=1000000)

reg.fit(library2_train, Cd_train)

Cd_pred = reg.predict(library2_test)
train_score = reg.score(library2_train, Cd_train)
test_score = reg.score(library2_test, Cd_test)
print('alpha = ', reg.alpha_)

print("train score: ",train_score)
print("test score: ",test_score)
print("----")

rmse_train = np.sqrt(mean_squared_error(Cd_train, reg.predict(library2_train)))
rmse_test = np.sqrt(mean_squared_error(Cd_test, reg.predict(library2_test)))
#percent_error = mean_absolute_percentage_error(Cd_test, reg.predict(library_test))
print("RMSE train: ",rmse_train)
print("RMSE test: ",rmse_test)
print("----")
print("----")

coefs = reg.coef_
learned_dict = list(zip(library2_names, coefs))
for i in range(len(learned_dict)):
    if np.abs(learned_dict[i][1]) > 0:
        print(learned_dict[i][0], learned_dict[i][1])

alpha =  100.0
train score:  0.999309696741288
test score:  0.9993971169022282
----
RMSE train:  105.93499448255572
RMSE test:  111.68928661353623
----
----
Re^-1phi^-1theta^0 11.162992544919673
Re^-1phi^-1theta^2 0.05315808479533798
Re^-1phi^-1theta^3 0.008079809766040995
Re^-1phi^0theta^0 12.285608877277346
Re^-1phi^0theta^1 0.2425302257838968
Re^-1phi^0theta^2 0.4321890150629319
Re^-1phi^1theta^1 0.4613017890114485


In [ ]:

# df = pd.read_csv(r'Library/Inclined_Library/inclined_ellipsoid_learned_dictionary.csv', usecols=[1], header=None)
# data = df.to_numpy()

# data = np.round(data, 3)

# print(data)

learned_dict = list(zip(dict, data))
for i in range(len(learned_dict)):
    if np.abs(learned_dict[i][1]) > 0:
        print(learned_dict[i][0], learned_dict[i][1])

Re^-1phi^-2theta^-1 [12.329]
Re^0phi^-2theta^-1 [12.604]
Re^-1phi^-1theta^-1 [0.498]


In [ ]:
theta_train

array([0.785398 , 0.       , 1.178097 , 1.5708   , 1.5708   , 1.5708   ,
       0.       , 1.5708   , 0.       , 0.3926991, 0.3926991, 0.3926991,
       1.5708   , 0.       , 1.5708   , 0.       , 1.178097 , 0.3926991,
       0.       , 1.178097 , 0.3926991, 0.3926991, 0.       , 0.785398 ,
       1.178097 , 0.       , 0.785398 , 1.5708   , 0.785398 , 0.3926991,
       0.3926991, 0.3926991, 1.178097 , 0.785398 , 0.785398 , 0.785398 ,
       0.785398 , 1.178097 , 0.785398 , 0.3926991, 0.785398 , 0.785398 ,
       0.3926991, 0.3926991, 1.5708   , 1.5708   , 1.178097 , 0.785398 ,
       1.178097 , 1.5708   , 1.5708   , 0.       , 0.3926991, 0.785398 ,
       0.       , 1.5708   , 0.       , 0.3926991, 0.       , 0.       ,
       1.5708   , 1.178097 , 1.178097 , 0.785398 , 0.       , 0.785398 ,
       0.785398 , 1.178097 , 0.       , 1.178097 , 1.178097 , 1.178097 ,
       0.       , 1.5708   , 0.       , 0.       , 0.785398 , 0.       ,
       0.       , 0.3926991, 1.5708   , 0.785398 , 